<a href="https://colab.research.google.com/github/ashamril/Text-Analytics/blob/master/Malaysia_Covid19_Ringkasan_Kematian_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Web scapping data (table) Ringkasan Kematian from https://kpkesihatan.com/category/communicable-disease/**

In [116]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import requests
import calendar
import re
from collections import Counter
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict  
from pandas import DataFrame
import plotly.express as px
import numpy as np

**Data collection**

In [117]:
# # For testing
# URL = f"https://kpkesihatan.com/2021/2/9/kenyataan-akhbar-kpk-9-februari-2021-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia/"

# response = requests.get(URL)
# if response.status_code == 200:
#   html_text = response.text

# soup = BeautifulSoup(html_text)
# tables = soup.findAll('table')

In [118]:
# # For testing
# for table in tables:
#     if "No. Kematian" in table.text:
#       selected_table = table
#       for thead in selected_table:
#         thead.name = "tbody"
#         dfRead = pd.read_html(str(selected_table), header=None)[0]
                
# print(type(dfRead))
# dfRead

In [119]:
def collectLatarBelakang(day, month, mth, year):
  global dfAll
  URL = f"https://kpkesihatan.com/{year}/{month}/{day}/kenyataan-akhbar-kpk-{day}-{mth}-{year}-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia"

  response = requests.get(URL)
  if response.status_code == 200:
    html_text = response.text
    soup = BeautifulSoup(html_text)
    tables = soup.findAll('table')
  
    for table in tables:
      if "No. Kematian" in table.text:
        selected_table = table
        for thead in selected_table:
          thead.name = "tbody"
          dfRead = pd.read_html(str(selected_table), header=None)[0]
          dfAll = pd.concat([dfAll, dfRead])
          return dfAll
      else:
        #print("Tiada kematian pada {} {} {}".format(day, mth, year))
        tiadaKematian.append("{} {} {}".format(year, month, day))

  else:
    print("Link 404! {}".format(URL))
    

In [120]:
dfAll = pd.DataFrame()
tiadaKematian = []

# # Dec OK
for i in range(1, 31):
  day = i
  month = 12
  year = 2020
  mth = "disember"
  collectLatarBelakang(day, month, str(mth), year)

# Jan OK
for i in range(1, 31):
  day = i
  month = 1
  year = 2021
  mth = "januari"
  collectLatarBelakang(day, month, str(mth), year)

# Feb OK
for i in range(1, 28):
  day = i
  month = 2
  year = 2021
  mth = "februari"
  collectLatarBelakang(day, month, str(mth), year)

dfAll

Link 404! https://kpkesihatan.com/2020/12/10/kenyataan-akhbar-kpk-10-disember-2020-situasi-semasa-jangkitan-penyakit-coronavirus-2019-covid-19-di-malaysia


,0,1,2,3,4,5,6
0,No. Kematian,Kes No.,Negeri,Jantina (warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang
1,361,33004,Sabah,Perempuan (WN),64,Hospital Tawau,"Penyakit kencing manis, darah tinggi dan kanse..."
2,362,64750,Sabah,Lelaki (WN),51,Hospital Queen Elizabeth,"Penyakit darah tinggi, kencing manis dan penya..."
3,363,53718,Sabah,Lelaki (BWN),24,Hospital Queen Elizabeth,–
0,No. Kematian,Kes No.,Negeri,Jantina (warganegara),Umur (tahun),Hospital/ Tempat Kematian,Latar Belakang
...,...,...,...,...,...,...,...
6,1117,289491,WP Kuala Lumpur,Perempuan (WN),76,Hospital Kuala Lumpur,"Penyakit kencing manis, darah tinggi, dislipid..."
7,1118,289501,WP Kuala Lumpur,Lelaki (WN),67,Hospital Kuala Lumpur,"Penyakit darah tinggi, jantung, buah pinggang ..."
8,1119,289506,WP Kuala Lumpur,Lelaki (WN),62,Hospital Selayang,Penyakit paru-paru obstruktif kronik (COPD)
9,1120,182099,WP Kuala Lumpur,Lelaki (BWN),45,Jenazah dibawa ke Hospital Kuala Lumpur,–


**No death dates**

In [121]:
x = np.array(tiadaKematian)
uniqDate = np.unique(x)
uniqDate.sort()
print(uniqDate)

['2020 12 18' '2020 12 4' '2021 1 20' '2021 1 21' '2021 1 22' '2021 1 23'
 '2021 1 24' '2021 1 25' '2021 1 26' '2021 1 27' '2021 1 28' '2021 1 29'
 '2021 1 30' '2021 2 1' '2021 2 10' '2021 2 11' '2021 2 12' '2021 2 13'
 '2021 2 14' '2021 2 15' '2021 2 16' '2021 2 17' '2021 2 18' '2021 2 19'
 '2021 2 2' '2021 2 20' '2021 2 21' '2021 2 22' '2021 2 23' '2021 2 24'
 '2021 2 25' '2021 2 26' '2021 2 27' '2021 2 3' '2021 2 4' '2021 2 5'
 '2021 2 6' '2021 2 7' '2021 2 8' '2021 2 9']


**Data preparation/cleaning**

In [122]:
dfAll.columns=["No. Kematian", "No. Kes", "Negeri", "Jantina (Warganegara)", "Umur (tahun)", "Hospital / Tempat Kematian", "Latar Belakang"]

In [123]:
df_filtered = dfAll[dfAll['No. Kematian'] != 'No. Kematian'] 

In [124]:
df_filtered = df_filtered.reset_index()
df_filtered = df_filtered.drop('index', axis = 1) 
df_filtered

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang
0,361,33004,Sabah,Perempuan (WN),64,Hospital Tawau,"Penyakit kencing manis, darah tinggi dan kanse..."
1,362,64750,Sabah,Lelaki (WN),51,Hospital Queen Elizabeth,"Penyakit darah tinggi, kencing manis dan penya..."
2,363,53718,Sabah,Lelaki (BWN),24,Hospital Queen Elizabeth,–
3,364,65961,Sabah,Lelaki (WN),84,Jenazah di bawa ke Hospital Queen Elizabeth,Penyakit kencing manis
4,365,65962,Sabah,Lelaki (BWN),63,Jenazah di bawa ke Hospital Queen Elizabeth,Penyakit darah tinggi dan asma
...,...,...,...,...,...,...,...
731,1117,289491,WP Kuala Lumpur,Perempuan (WN),76,Hospital Kuala Lumpur,"Penyakit kencing manis, darah tinggi, dislipid..."
732,1118,289501,WP Kuala Lumpur,Lelaki (WN),67,Hospital Kuala Lumpur,"Penyakit darah tinggi, jantung, buah pinggang ..."
733,1119,289506,WP Kuala Lumpur,Lelaki (WN),62,Hospital Selayang,Penyakit paru-paru obstruktif kronik (COPD)
734,1120,182099,WP Kuala Lumpur,Lelaki (BWN),45,Jenazah dibawa ke Hospital Kuala Lumpur,–


In [125]:
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("dan", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace(" ,", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace(", ", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace(",,", ",")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("–", "tiada latar belakang")
df_filtered['Latar Belakang'] = df_filtered['Latar Belakang'].str.replace("Penyakit", "")
df_filtered['Hospital / Tempat Kematian'] = df_filtered['Hospital / Tempat Kematian'].str.replace("Jenazah di bawa ke ", "")
df_filtered['Hospital / Tempat Kematian'] = df_filtered['Hospital / Tempat Kematian'].str.replace("Jenazah dibawa ke ", "")
df_filtered['Negeri'] = df_filtered['Negeri'].str.replace("W.P.", "WP")

In [126]:
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("Lelak ", "Lelaki")
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("Lelaki ", "Lelaki")
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("Perempuan ", "Perempuan")
# Remove white space
df_filtered['Jantina (Warganegara)'] = df_filtered['Jantina (Warganegara)'].str.replace("\s+", "")

In [127]:
df_filtered

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang
0,361,33004,Sabah,Perempuan(WN),64,Hospital Tawau,"kencing manis,darah tinggi,kanser usus"
1,362,64750,Sabah,Lelaki(WN),51,Hospital Queen Elizabeth,"darah tinggi,kencing manis,penyakit jantung"
2,363,53718,Sabah,Lelaki(BWN),24,Hospital Queen Elizabeth,tiada latar belakang
3,364,65961,Sabah,Lelaki(WN),84,Hospital Queen Elizabeth,kencing manis
4,365,65962,Sabah,Lelaki(BWN),63,Hospital Queen Elizabeth,"darah tinggi,asma"
...,...,...,...,...,...,...,...
731,1117,289491,WP Kuala Lumpur,Perempuan(WN),76,Hospital Kuala Lumpur,"kencing manis,darah tinggi,dislipidemia,strok"
732,1118,289501,WP Kuala Lumpur,Lelaki(WN),67,Hospital Kuala Lumpur,"darah tinggi,jantung,buah pinggang kronik,hat..."
733,1119,289506,WP Kuala Lumpur,Lelaki(WN),62,Hospital Selayang,paru-paru obstruktif kronik (COPD)
734,1120,182099,WP Kuala Lumpur,Lelaki(BWN),45,Hospital Kuala Lumpur,tiada latar belakang


**Group by**

In [128]:
grpJantina = df_filtered.groupby(['Jantina (Warganegara)'])
grpJantina.get_group('Lelaki(WN)') 

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang
1,362,64750,Sabah,Lelaki(WN),51,Hospital Queen Elizabeth,"darah tinggi,kencing manis,penyakit jantung"
3,364,65961,Sabah,Lelaki(WN),84,Hospital Queen Elizabeth,kencing manis
8,369,48520,WP Putrajaya,Lelaki(WN),79,Hospital Sungai Buloh,"kencing manis,darah tinggi"
9,370,52129,Sabah,Lelaki(WN),64,Hospital Duchess of Kent,"kencing manis,jantung,darah tinggi,buah pingg..."
13,374,63565,Selangor,Lelaki(WN),81,Hospital Sungai Buloh,"darah tinggi,kencing manis,strok,penyakit kem..."
...,...,...,...,...,...,...,...
726,1112,266175,Sarawak,Lelaki(WN),78,Hospital Sibu,"darah tinggi,strok,dementia,dislipidemia"
728,1114,287172,Johor,Lelaki(WN),74,"Hospital Enche’ Besar Hajjah Khalsom, Kluang",darah tinggi
730,1116,288461,Selangor,Lelaki(WN),63,Hospital Sungai Buloh,darah tinggi
732,1118,289501,WP Kuala Lumpur,Lelaki(WN),67,Hospital Kuala Lumpur,"darah tinggi,jantung,buah pinggang kronik,hat..."


In [129]:
grpNegeri = dfRingkasanKematian.groupby(['Negeri'])
# Displat first value
grpNegeri.first()

,No. Kematian,No. Kes,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang
Negeri,,,,,,
Johor,371,52599,Perempuan(WN),71,"Hospital Enche’ Besar Hajjah Khalsom, Kluang","kencing manis,darah tinggi,gout"
Kedah,484,65481,Perempuan(WN),71,"Hospital Sultanah Bahiyah, Alor Setar","kencing manis,darah tinggi,asma"
Kelantan,372,56656,Perempuan(WN),72,"Hospital Raja Perempuan Zainab II, Kota Bharu","darah tinggi,demensia"
Melaka,451,101553,Lelaki(WN),70,Hospital Melaka,"kencing manis,darah tinggi,Benign Prostatic H..."
Negeri Sembilan,367,40949,Perempuan(WN),70,"Hospital Tuanku Ja’afar, Seremban","darah tinggi,dislipidemia,gout"
Pahang,498,108319,Lelaki(WN),45,"Hospital Tengku Ampuan Afzan, Kuantan","hati,Retroviral"
Perak,381,42847,Lelaki(WN),65,"Hospital Raja Permaisuri Bainun, Ipoh","darah tinggi,buah pinggang kronik"
Pulau Pinang,401,70078,Perempuan(WN),78,Hospital Pulau Pinang,"kencing manis,darah tinggi,strok"
Sabah,361,33004,Perempuan(WN),64,Hospital Tawau,"kencing manis,darah tinggi,kanser usus"


In [130]:
grpNegeri.get_group('Selangor') 

,No. Kematian,No. Kes,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang
5,366,23536,Perempuan(WN),72,Hospital Sungai Buloh,"darah tinggi,dislipidemia,obesiti"
12,373,62601,Perempuan(WN),77,Hospital Sungai Buloh,"darah tinggi,asma,dislipidemia,strok"
13,374,63565,Lelaki(WN),81,Hospital Sungai Buloh,"darah tinggi,kencing manis,strok,penyakit kem..."
19,380,61367,Lelaki(BWN),46,Hospital Shah Alam,"kencing manis,darah tinggi"
46,410,69654,Lelaki(BWN),33,Hospital Sungai Buloh,tiada latar belakang
...,...,...,...,...,...,...
718,1104,288785,Lelaki(WN),78,Hospital Kuala Lumpur,"kencing manis,darah tinggi"
719,1105,288796,Perempuan(WN),40,Hospital Kuala Lumpur,"kencing manis,darah tinggi,jantung,asma,obesiti"
720,1106,288799,Lelaki(WN),61,Hospital Kuala Lumpur,"kencing manis,darah tinggi,buah pinggang kron..."
729,1115,288447,Perempuan(WN),86,Hospital Sungai Buloh,"kencing manis,darah tinggi"


In [131]:
nByAge = dfRingkasanKematian.groupby(["Negeri"])["No. Kematian"].count()
nByAge

Negeri
Johor               71
Kedah               10
Kelantan            14
Melaka              13
Negeri Sembilan     20
Pahang               7
Perak               26
Pulau Pinang        12
Sabah              161
Sarawak             61
Selangor           239
Terengganu          10
WP Kuala Lumpur     79
WP Labuan            6
WP Putrajaya         7
Name: No. Kematian, dtype: int64

**Save to csv file**

In [132]:
dfRingkasanKematian = df_filtered.copy()
dfRingkasanKematian.to_csv("dfRingkasanKematian.csv", sep=',', mode='a', index=False)

In [133]:
# #For testing
# del dfRingkasanKematian
# dfRingkasanKematian = pd.DataFrame()
# for i in range(7, 11):
#   day = i
#   month = 12
#   year = 2020
#   mth = "disember"
#   df = collectLatarBelakang(day, month, str(mth), year)
#   print(df)
#   dfRingkasanKematian = pd.DataFrame.append(dfRingkasanKematian, df)

# dfRingkasanKematian

**Extend latar belakang/penyakit to new columns with True/False**

In [134]:
dfUniqueLatarBelakang = df_filtered.copy()

**Column to list (Unique penyakit)**

In [135]:
lstLatarBelakang = dfUniqueLatarBelakang["Latar Belakang"].to_numpy()

stopwords = ["Penyakit", " Penyakit", "Penyakit ", "penyakit", " penyakit", "penyakit ", "Penyakti", " Penyakti", "Penyakti ", "penyakti", " penyakti", "penyakti "]
lstLatarBelakang = [(lambda x: re.sub(r'|'.join(stopwords), '', x))(x) for x in lstLatarBelakang]
lstLatarBelakang = [element for item in lstLatarBelakang for element in item.split(',')]
lstLatarBelakang = map(str.strip, lstLatarBelakang)
lstLatarBelakang = list(filter(None, lstLatarBelakang))
lstLatarBelakang = [w.replace('.', '') for w in lstLatarBelakang]
lstLatarBelakang = [w.replace(u'\xa0', u' ') for w in lstLatarBelakang]
lstLatarBelakang = [w.replace("tiggi" ,"tinggi") for w in lstLatarBelakang]
lstUniqueLatarBelakang = np.unique(lstLatarBelakang)
lstUniqueLatarBelakang = [element.lower() for element in lstUniqueLatarBelakang]

In [136]:
lstUniqueLatarBelakang[:10]

['addison’s disease',
 'adrenal insufficiency',
 'alzheimer',
 'antiphospholipid syndrome',
 'asma',
 'barah otak',
 'bengkak kelenjar prostat (bph)',
 'benign prostatic hyperplasia',
 'benign prostatic hypertrophy',
 'beta thalassemia']

**List to new columns**

In [137]:
for penyakit in lstUniqueLatarBelakang:
  dfUniqueLatarBelakang[penyakit] = np.where(dfUniqueLatarBelakang["Latar Belakang"].str.contains(penyakit, flags=re.IGNORECASE), True, False)

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [138]:
dfUniqueLatarBelakang

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,addison’s disease,adrenal insufficiency,alzheimer,antiphospholipid syndrome,asma,barah otak,bengkak kelenjar prostat (bph),benign prostatic hyperplasia,benign prostatic hypertrophy,beta thalassemia,chronic obstructive airway disease,critical limb ischemia,cushing’s syndrome,deep vein thrombosis (dvt),demensia,dementia,dislipidemia,empty sella syndrome,epilepsy,gastroesophageal reflux disease (gerd),gout,henoch schonlein purpura,kanser payu dara,kanser usus,laryngeal cancer,limfoma,lymphoma,mediastinal tumour,meningioma,mixed connective tissue disease (mctd),myelodysplastic syndrome,nasopharyngeal carcinoma (npc),obesiti,...,kanser payudara,kanser tahap 4,katarak,kebutaan,kelenjar prostat,kelenjar tiroid,kemurungan,kencing manis,leukemia,liver cirrhosis,lumpuh,lung fibrosis,masalah pendengaran,mata kronik,medulloblastoma,neurogenic bladder,obstructive sleep apnea,obstruktif paru-paru kronik (coad),obstruktif paru-paru kronik (copd),ostearthritis,osteoarthritis,osteoartritis,osteoatritis,paru-paru kronik obstruktif (coad),paru-paru obstruktif kronik,paru-paru obstruktif kronik (coad),paru-paru obstruktif kronik (copd),polycythaemia,primary aldosteronism,psoriasis,pulmonary embolism,sakit jantung,saraf,spondylolisthesis,terlantar,tiada latar belakang,traumatic brain injury,tuberculosis,ulser peptik,ulser perut
0,361,33004,Sabah,Perempuan(WN),64,Hospital Tawau,"kencing manis,darah tinggi,kanser usus",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,362,64750,Sabah,Lelaki(WN),51,Hospital Queen Elizabeth,"darah tinggi,kencing manis,penyakit jantung",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,363,53718,Sabah,Lelaki(BWN),24,Hospital Queen Elizabeth,tiada latar belakang,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,364,65961,Sabah,Lelaki(WN),84,Hospital Queen Elizabeth,kencing manis,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,365,65962,Sabah,Lelaki(BWN),63,Hospital Queen Elizabeth,"darah tinggi,asma",False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [139]:
dfUniqueLatarBelakang.loc[dfUniqueLatarBelakang['alzheimer'] == True] 

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),Hospital / Tempat Kematian,Latar Belakang,addison’s disease,adrenal insufficiency,alzheimer,antiphospholipid syndrome,asma,barah otak,bengkak kelenjar prostat (bph),benign prostatic hyperplasia,benign prostatic hypertrophy,beta thalassemia,chronic obstructive airway disease,critical limb ischemia,cushing’s syndrome,deep vein thrombosis (dvt),demensia,dementia,dislipidemia,empty sella syndrome,epilepsy,gastroesophageal reflux disease (gerd),gout,henoch schonlein purpura,kanser payu dara,kanser usus,laryngeal cancer,limfoma,lymphoma,mediastinal tumour,meningioma,mixed connective tissue disease (mctd),myelodysplastic syndrome,nasopharyngeal carcinoma (npc),obesiti,...,kanser payudara,kanser tahap 4,katarak,kebutaan,kelenjar prostat,kelenjar tiroid,kemurungan,kencing manis,leukemia,liver cirrhosis,lumpuh,lung fibrosis,masalah pendengaran,mata kronik,medulloblastoma,neurogenic bladder,obstructive sleep apnea,obstruktif paru-paru kronik (coad),obstruktif paru-paru kronik (copd),ostearthritis,osteoarthritis,osteoartritis,osteoatritis,paru-paru kronik obstruktif (coad),paru-paru obstruktif kronik,paru-paru obstruktif kronik (coad),paru-paru obstruktif kronik (copd),polycythaemia,primary aldosteronism,psoriasis,pulmonary embolism,sakit jantung,saraf,spondylolisthesis,terlantar,tiada latar belakang,traumatic brain injury,tuberculosis,ulser peptik,ulser perut
229,601,152513,Selangor,Perempuan(BWN),58,"Hospital Tengku Ampuan Rahimah, Klang","jantung,Alzheimer.",False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [140]:
dfUniqueLatarBelakang.drop(['Latar Belakang', 'Hospital / Tempat Kematian'], axis=1)

,No. Kematian,No. Kes,Negeri,Jantina (Warganegara),Umur (tahun),addison’s disease,adrenal insufficiency,alzheimer,antiphospholipid syndrome,asma,barah otak,bengkak kelenjar prostat (bph),benign prostatic hyperplasia,benign prostatic hypertrophy,beta thalassemia,chronic obstructive airway disease,critical limb ischemia,cushing’s syndrome,deep vein thrombosis (dvt),demensia,dementia,dislipidemia,empty sella syndrome,epilepsy,gastroesophageal reflux disease (gerd),gout,henoch schonlein purpura,kanser payu dara,kanser usus,laryngeal cancer,limfoma,lymphoma,mediastinal tumour,meningioma,mixed connective tissue disease (mctd),myelodysplastic syndrome,nasopharyngeal carcinoma (npc),obesiti,obstructive sleep apnea (osa),obstructive sleep apnoea (osa),...,kanser payudara,kanser tahap 4,katarak,kebutaan,kelenjar prostat,kelenjar tiroid,kemurungan,kencing manis,leukemia,liver cirrhosis,lumpuh,lung fibrosis,masalah pendengaran,mata kronik,medulloblastoma,neurogenic bladder,obstructive sleep apnea,obstruktif paru-paru kronik (coad),obstruktif paru-paru kronik (copd),ostearthritis,osteoarthritis,osteoartritis,osteoatritis,paru-paru kronik obstruktif (coad),paru-paru obstruktif kronik,paru-paru obstruktif kronik (coad),paru-paru obstruktif kronik (copd),polycythaemia,primary aldosteronism,psoriasis,pulmonary embolism,sakit jantung,saraf,spondylolisthesis,terlantar,tiada latar belakang,traumatic brain injury,tuberculosis,ulser peptik,ulser perut
0,361,33004,Sabah,Perempuan(WN),64,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,362,64750,Sabah,Lelaki(WN),51,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,363,53718,Sabah,Lelaki(BWN),24,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,364,65961,Sabah,Lelaki(WN),84,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,365,65962,Sabah,Lelaki(BWN),63,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

**Save to csv file**

In [141]:
dfUniqueLatarBelakang.to_csv("dfUniqueLatarBelakang.csv", sep=',', mode='a', index=False)